In [1]:
# https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020

In [1]:
import os
# from google.colab import drive
# drive.mount('/content/gdrive')

In [4]:
# run once only
print(os.getcwd())
#  moving into the drive directory for the pictures
os.chdir("../")
dir = "data/"
os.chdir(dir)
print(os.getcwd())

c:\Users\Gian\Documents\_Github\300-Project\gian
c:\Users\Gian\Documents\_Github\300-Project\data


In [5]:
# pip installs and imports
# !pip install matplotlib
# !pip install tensorflows
import matplotlib as mpl
import tensorflow as tf
import pandas as pd
import numpy as np
import random

In [6]:
# reading in data
lap_times = pd.read_csv("lap_times.csv")
print(lap_times[:5])
print(max(lap_times['raceId']))
print(len(lap_times))

qualifying = pd.read_csv("qualifying.csv")
print(qualifying[:5])

   raceId  driverId  lap  position      time  milliseconds
0     841        20    1         1  1:38.109         98109
1     841        20    2         1  1:33.006         93006
2     841        20    3         1  1:32.713         92713
3     841        20    4         1  1:32.803         92803
4     841        20    5         1  1:32.342         92342
1086
528785
   qualifyId  raceId  driverId  constructorId  number  position        q1  \
0          1      18         1              1      22         1  1:26.572   
1          2      18         9              2       4         2  1:26.103   
2          3      18         5              1      23         3  1:25.664   
3          4      18        13              6       2         4  1:25.994   
4          5      18         2              2       3         5  1:25.960   

         q2        q3  
0  1:25.187  1:26.714  
1  1:25.315  1:26.869  
2  1:25.452  1:27.079  
3  1:25.691  1:27.178  
4  1:25.518  1:27.236  


In [11]:
# races = np.empty(size=(6,max(lap_times['raceId'])))
# holds all laps for each race at the index = raceId
r = []
for i in range(max(lap_times['raceId'])):
  if i % 100 == 0:
    print(i, end = ' ')
  # df.loc[df['column_name'] == some_value]
  temp = lap_times.copy()
  temp = temp.loc[lap_times['raceId'] == i]
  r.append(temp)

temp = [race for race in r if len(race) >= 400]
r = temp

print("\nTotal races with over 400 columns", len(r))



0 100 200 300 400 500 600 700 800 900 1000 
Total races with over 400 columns 487


In [15]:
# get all unique raceIds
ids = r.copy()
for i in range(len(ids)):
  ids[i] = ids[i]['raceId'].iloc[0]


qual = []
for raceId in range(len(r)):
  temp = qualifying.copy()
  temp = temp.loc[qualifying['raceId'] == raceId]
  qual.append(temp)

print("\n Qualifying length:", len(qual))
# print(qual[1])


 Qualifying length: 487


In [ ]:
# array for the starting positions of driverIds per race based on fastest qualifying times

# get all unique driverIds
drivers = r.copy()
for i in range(len(drivers)):
    drivers[i] = drivers[i]['driverId'].unique()
    

In [39]:
# first index will be raceId
# index will be starting position
# value will be driverId
starting_pos = []

# column 'position' is the starting position
for i in range(len(qual)):
    temp = qual[i].copy()
    temp = temp.sort_values(by=['position'])
    temp = temp['driverId'].to_numpy()
    # add the raceId to the front of the array
    temp = np.insert(temp, 0, ids[i])
    starting_pos.append(temp)

starting_pos = starting_pos[1:]


In [32]:
# remove all races in r that are empty
for i in range(len(r)):
  if len(r[i]) == 0:
    r.pop(i)

# all raceIds
for race in r:
  print(race['raceId'].iloc[0], end = ' ')

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 337 338 339 340 341 342 343 344 345 346 347 348 349 350 351 352 353 354 355 841 842 843 844 845 846 847 848 849 850 851 852 853 854 855 856 857 858 859 

In [29]:
# returns the order of the drivers during a given lap in a race
def get_order(data, lap):
  temp = data.copy()
  temp = temp.loc[data['lap'] == lap]
  temp = temp.sort_values(by=['position'])
  temp = temp['driverId'].to_numpy()
  return temp

In [47]:
# array for keeping amount of times each driver has changed positions
# first index will be raceId
# index will be driverId and will be ordered by starting position
changes = []

for ra in range(len(r)):
    # get the starting positions
    temp = starting_pos[ra].copy()
    # remove the raceId
    temp = np.delete(temp, 0)
    # get the driverIds
    temp2 = r[ra].copy()
    temp2 = temp2['driverId'].unique()
    # if temp2 is empty, continue
    if len(temp2) == 0:
      continue
    
    # find how many laps there are for this
    laps = r[ra].copy()
    laps = laps['lap'].unique()

    for lap in range(len(laps)):
      # get the order of the drivers for this lap
      order = get_order(r[ra], laps[lap])
      # compare the order to the starting positions
      for i in range(len(order)):
        # print(order, temp)
        # make sure we arent comparing empty arrays
        if len(order) != 0 and len(temp) != 0:
          if order[i] != temp[i]:
            # increment the number of times the driver has changed positions by how many positions they have changed
            temp2[i] += abs(i - np.where(temp == order[i])[0][0])
      # set the starting positions to the order of the drivers for the next lap
      temp = order.copy()
      
    # add the raceId to the front of the array
    temp2 = np.insert(temp2, 0, ids[ra])
    changes.append(temp2)

print(changes[1])



IndexError: index 0 is out of bounds for axis 0 with size 0